# Merge USGS Terrain COGs
Terrain data at 1 arc second (~30m) resolution is available from USGS as 32-bit cloud-optimized geotiff in a public AWS bucket.  Each file contains a 1x1 degree square (plus a small halo region which we will clip to remove).  

In [ ]:
import rioxarray 
from rioxarray.merge import merge_arrays
import os
import hvplot.xarray

In [ ]:
urls = ['https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n41w080/USGS_1_n41w080.tif',
 'https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n42w080/USGS_1_n42w080.tif',
 'https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n41w079/USGS_1_n41w079.tif',
 'https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n42w079/USGS_1_n42w079.tif']

Use GDAL settings from [COG Best Practices](https://github.com/pangeo-data/cog-best-practices/blob/main/0-single-cog.ipynb)

In [ ]:
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR' #This is KEY! otherwise we send a bunch of HTTP GET requests to test for common sidecar metadata
os.environ['AWS_NO_SIGN_REQUEST']='YES' #Since this is a public bucket, we don't need authentication
os.environ['GDAL_MAX_RAW_BLOCK_CACHE_SIZE']='200000000'  #200MB: Want this to be greater than size of uncompressed raster to overcome a 10 MB limit in the GeoTIFF driver for range request merging.
os.environ['GDAL_SWATH_SIZE']='200000000'  #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE
os.environ['VSI_CURL_CACHE_SIZE']='200000000' #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE

Get full resolution data (not an overview), remove the band dimension, then clip to actual degree square, removing the extra data that surrounds each square:

In [ ]:
def clip(da):
    x = da['x'].values; x0 = x.min().round(); x1 = x.max().round()
    y = da['y'].values; y0 = y.min().round(); y1 = y.max().round()
    return da.sel(x=slice(x0,x1), y=slice(y1,y0))

In [ ]:
1/3600/2

In [ ]:
darrays = []
for url in urls:
    da = rioxarray.open_rasterio(url, masked=True, chunks=True)
    #da = da.squeeze('band') 
    darrays.append(clip(da))                 

In [ ]:
da = merge_arrays(darrays)

In [ ]:
import numpy as np

In [ ]:
da.hvplot.image(x='x', y='y', geo=True, rasterize=True, 
                frame_width=500, cmap='rainbow', tiles='ESRI')

In [ ]:
import numpy as np
np.diff(np.unique(np.diff(da['x'].values)))

#### Explore the full list of available degree squares

In [ ]:
fs = fsspec.filesystem('s3', anon=True)

In [ ]:
flist = fs.glob('prd-tnm/StagedProducts/Elevation/1/TIFF/')

In [ ]:
flist[:5]

In [ ]:
flist[-5:]